In [1]:
import numpy as np
import pandas as pd
import glob
import ipyparallel as p
import os
import ntpath
import ProgressBar as pb
from pymail import alert
import time

In [2]:
exname = 'smex8_sen'
method = '8'
site = 'P301'

In [3]:
files = glob.glob('/Volumes/data/RHESSys_out/P301/%s/*_basin.daily'%exname)

In [4]:
len(files)

150000

In [5]:
params = pd.read_pickle('./data/soil_params_multispinup_%s_smex%s_sen.pcl'%(site,method))

In [6]:
files = pd.DataFrame({'basin_daily':files})

In [7]:
def getIDX(fl):
    return int(fl.split('_')[-2])

In [8]:
files['scenidx'] = files.basin_daily.map(getIDX)

In [9]:
params = pd.merge(params,files, on= 'scenidx')

In [10]:
from rhessys import snowmelt_experiment as smex

In [11]:
params['Tnosnow'] = params.TpeakSWE+np.ceil(params.PeakSWE/params.SMR)

In [12]:
params['date_peakSWE'] = params.apply(smex.DOPEAKSWE2cal,axis=1)
params['date_ONS'] =params.apply(smex.DONS2cal,axis=1)

In [13]:
c = p.Client()
view = c.load_balanced_view()

In [14]:
c.ids

[0, 1, 2, 3, 4, 5, 6, 7]

In [15]:
len(params)

150000

In [16]:
dr = pd.date_range('2008-10-1','2009-10-02',freq='D')

In [17]:
%px dr = pd.date_range('2008-10-1','2009-10-02',freq='D')

In [18]:
def processSM(bd,idx,strt,nd):

    # read in the basin daily data
    data = pd.read_table(bd,delimiter=' ')

    data.index = pd.DatetimeIndex(dr)

    data['ET'] = data.evap + data.trans

    ET = data.ET.sum() # compute total ET
    Q = data.streamflow.sum() # compute total streamflow 
    Qbf = data.baseflow.sum() # compute total baseflow
    Qrf = data['return'].sum() # compute total returnflow,p
    sm = data.snowmelt.sum()
    rech = data.recharge.sum() # compute total recharge
    sat = data['%sat_area'].mean() # compute the mean saturated area
    p = data.precip.sum() # compute water year precip
    rz_storage = data.rz_storage.mean() # root zone storage
    rz_drainage = data.rz_drainage.mean() # root zone drainage 
    unsat_drainage = data.unsat_drain.mean() # unsaturated zone drainage
    unsat_storage = data.unsat_stor.mean() # unsaturated zone drainage
    sat_def = data.sat_def.mean() # saturation deficite (mm)
    sat_def_z = data.sat_def_z.mean() # saturation deficite depth, is this the depth to ground water?
    gwStore = data['gw.storage'].sum() # gw store
    gwFlux = data['gw.Qout'].mean()
    ETRate = data.ET.mean()
    
    # create full output object
    full = (idx,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,
            unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate)
    
    data = data.loc[strt:nd] # crop the data to the melt season
    
    ET = data.ET.sum() # compute total ET
    Q = data.streamflow.sum() # compute total streamflow 
    Qbf = data.baseflow.sum() # compute total baseflow
    Qrf = data['return'].sum() # compute total returnflow,p
    sm = data.snowmelt.sum()
    rech = data.recharge.sum() # compute total recharge
    sat = data['%sat_area'].mean() # compute the mean saturated area
    p = data.precip.sum() # compute water year precip
    rz_storage = data.rz_storage.mean() # root zone storage
    rz_drainage = data.rz_drainage.mean() # root zone drainage 
    unsat_drainage = data.unsat_drain.mean() # unsaturated zone drainage
    unsat_storage = data.unsat_stor.mean() # unsaturated zone drainage
    sat_def = data.sat_def.mean() # saturation deficite (mm)
    sat_def_z = data.sat_def_z.mean() # saturation deficite depth, is this the depth to ground water?
    gwStore = data['gw.storage'].sum() # gw store
    gwFlux = data['gw.Qout'].mean()
    ETRate = data.ET.mean()
    
    melt = (idx,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,
            unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate)
    
    return full,melt

In [19]:
idx = 0
print params.basin_daily[idx]
processSM(params.basin_daily[idx],params.scenidx[idx],params.date_peakSWE[idx],params.date_ONS[idx])

/Volumes/data/RHESSys_out/P301/smex8_sen/smex8_sen_0_basin.daily


((0,
  116.65342500000004,
  616.4584529999998,
  230.1585649999999,
  386.29988800000007,
  558.6806300000001,
  0.0,
  716.4708499999999,
  2.2134157356948063,
  2.732354108991829,
  1941.0120784932024,
  4.322964885558577,
  1946.6444373569475,
  4011.1040799999773,
  388.1430449999999,
  11673.876054000002,
  0.2587964850136241,
  0.3178567438692099),
 (0,
  9.229027999999998,
  68.89143899999999,
  31.740791000000005,
  37.15065,
  61.962257,
  0.0,
  0.0,
  2.984500210526316,
  4.052827631578946,
  1942.07875531579,
  6.368756921052633,
  1946.6443399999994,
  4011.10408,
  70.650502,
  1366.2957949999998,
  0.29263349999999994,
  0.2428691578947368))

In [ ]:
#res = view.map(processSM,params.basin_daily[0:1000],params.scenidx[0:1000],params.date_peakSWE[0:1000],params.date_ONS[0:1000])
res = view.map(processSM,params.basin_daily,params.scenidx,params.date_peakSWE,params.date_ONS)

In [ ]:
prog = pb.ProgressBar(len(params))

In [ ]:
while res.ready() == False:
    prog.animate_ipython(res.progress)
    time.sleep(2)

#alert.send_alert('barnhatb@colorado.edu','Processing Nr1 %s has finished'%exname,'Your script has finished')

In [ ]:
res.done()

In [ ]:
res.progress

In [ ]:
full,melt = zip(*res.result())

## Merge processing results with parameters dataset

In [ ]:
idx,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate = zip(*full)

params = pd.read_pickle('./data/soil_params_multispinup_%s_smex%s_sen.pcl'%(site,method))

# make a date frame of the results
proc = pd.DataFrame()

arrs = [idx,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,
        sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate]
keys = ['scenidx','ET','Q','Qbf','Qrf','rech','sat','p','rz_storage','rz_drainage',
        'unsat_storage','unsat_drainage','sat_def','sat_def_z','sm','gwStore','gwFlux','ETRate']

for arr,key in zip(arrs,keys):
    proc[key] = arr # insert the results

params = pd.merge(params,proc,on='scenidx') # join on file index number
params.dropna(inplace=True) # drop missing values

params.to_hdf('./data/%s_%s_full.hdf'%(site,exname),'df',format='fixed',complevel=5,complib='bzip2',fletcher32=True)

In [ ]:
idx,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate = zip(*melt)

params = pd.read_pickle('./data/soil_params_multispinup_%s_smex%s_sen.pcl'%(site,method))

# make a date frame of the results
proc = pd.DataFrame()

arrs = [idx,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,
        sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate]
keys = ['scenidx','ET','Q','Qbf','Qrf','rech','sat','p','rz_storage','rz_drainage',
        'unsat_storage','unsat_drainage','sat_def','sat_def_z','sm','gwStore','gwFlux','ETRate']

for arr,key in zip(arrs,keys):
    proc[key] = arr # insert the results

params = pd.merge(params,proc,on='scenidx') # join on file index number
params.dropna(inplace=True) # drop missing values

params.to_hdf('./data/%s_%s_melt.hdf'%(site,exname),'df',format='fixed',complevel=5,complib='bzip2',fletcher32=True) # 

In [ ]:
alert.send_alert('barnhatb@colorado.edu','Processing P301 %s has finished'%exname,'Your script has finished')